<a href="https://colab.research.google.com/github/MoritzLaurer/rag-demo/blob/master/rag_langchain_ai_law.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Evaluating a RAG pipeline with LangChain and Hugging Face Endpoints or OpenAI

This notebook provides a quick demo for creating and evaluating a Retrieval Augmented Generation (RAG) pipeline with LangChain and Hugging Face Endpoints or OpenAI.

The demo has the following main steps:
1. Create an example vector database: The demo downloads 440 position paper PDFs which stakeholders had submitted to the EU public consultation on the EU White Paper on AI in 2020. These PDFs are processed and ingested in a vector database.
2. We then automatically generate questions about a sample of the texts with an LLM
3. Then we create a RAG pipeline and feed the generated questions into the RAG pipeline as user queries
4. RAG evaluation:
  - Retriever quality: If we ask a generated question to the RAG pipeline, does the pipeline's retriever retrieve the same original text which was used to generated the question? This provides an indication of retriever (and reranker) quality. This indicator is, however, imperfect, as the retriever could also retrieve other texts that help the RAG pipeline generate good answers beyond only the original text used for generating the question.
  - Answer quality: We also use an LLM to evaluate answer quality more broadly.


## Install packages

In [1]:
%%bash
pip install --upgrade pip -q
pip install langchain~=0.0.352
pip install langchainhub~=0.1.14
pip install openai~=1.6.0
pip install tiktoken~=0.5.2
pip install transformers>=4.35.2
pip install huggingface_hub~=0.20.1
pip install sentence_transformers~=2.2.2
pip install qdrant-client~=1.7.0
pip install PyMuPDF~=1.23.7


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.3/205.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
   ━━━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.15.1 requires protobuf<4.24,>=3.19.6, but you have protobuf 4.25.1 which is incompatible.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobu

In [2]:
import os
from google.colab import userdata
from huggingface_hub import login

# for using hugging face models
login(token=userdata.get('HF_TOKEN'))

# for using OAI models
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_KEY')

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Prepare example data

#### Download PDF data

In [3]:
## download PDF data
import os
import zipfile
import requests
from io import BytesIO

# URL of the zip file in your GitHub repo (make sure it's the raw file URL)
zip_url = 'https://github.com/MoritzLaurer/rag-demo/blob/master/data/position-papers-pdfs.zip?raw=true'

# Download the zip file
print("Downloading zip file...")
response = requests.get(zip_url)
zip_content = BytesIO(response.content)

# Define the extraction path
extract_path = '/content/data'

# Create directory if it doesn't exist
if not os.path.exists(extract_path):
    os.makedirs(extract_path)

# Extract the zip file
print("Extracting zip file...")
with zipfile.ZipFile(zip_content, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction completed.")

file_paths = [f for f in os.listdir(extract_path) if os.path.isfile(os.path.join(extract_path, f))]
print(f"{len(file_paths)} PDF files downloaded.")


Extracting zip file...
Extraction completed.
440 PDF files downloaded.


### Process data

In [4]:
from langchain.document_loaders import PDFMinerLoader
from langchain.document_loaders import PyMuPDFLoader
from tqdm.notebook import tqdm

directory = "./data"

docs = []
for pdf_path in tqdm(os.listdir(directory)):
  try:
    docs.append(PyMuPDFLoader(os.path.join(directory, pdf_path)).load())
  except Exception as e:
    print("Exception: ", e)


  0%|          | 0/440 [00:00<?, ?it/s]

Exception:  cannot open broken document
Exception:  cannot open broken document


In [5]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

"""text_splitter = CharacterTextSplitter(
    separator = " ",
    chunk_size = 1000,
    chunk_overlap  = 30,
    length_function = len,
    is_separator_regex = False,
)"""

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)

docs_processed = [text_splitter.split_documents(doc) for doc in docs]

docs_processed = [item for sublist in docs_processed for item in sublist]
print(len(docs_processed))

docs_processed[:1]

16932


[Document(page_content='EC White Paper: Consultation Response \nJune 2020 \n© 2020, Loughborough University, UKRI Project REF: ES/S010416/1  \n \n1 of 26 \nResponse \nto \nthe \nEuropean \nCommission’s \nConsultation on Artificial Intelligence: A European \napproach to excellence and trust \nThis document is a response to the European Commission’s Consultation on Artificial Intelligence, from \nLoughborough University systems engineering researchers Dr Melanie King and Paul Timms, written as part \nof the TECHNGI Academic Research Project (UKRI Project Ref: ES/S010416/1).  TECHNGI (Technology \nDriven Next Generation Insurance) is a cross-disciplinary research project investigating the opportunities and \nchallenges for the UK insurance industry arising from the application of new AI technologies, including machine \nlearning, distributed ledger, automated processing, and the explosion of available dataa. \nWe provide both general comments on the white paper [1], and address more speci

#### Sample data to reduce embedding and generation costs

In [6]:
import random
random.seed(42)

# sample corpus for embedding
index_random = random.sample(range(len(docs_processed)), 100)
docs_processed_samp = [docs_processed[i] for i in index_random]

# sample some contexts generate questions from
docs_processed_for_q_generation = docs_processed_samp[:5]


## Automatic question generation for evaluation

This section generates questions which users could ask about a specific text in the database. This allows us to assess:  
- If we ask a generated question to the RAG pipeline, does the pipeline's retriever retrieve the same text which was used to generated the question? This provides an indication of retriever (and reranker) quality.
- Beyond the original text used for generating the question, the retriever might retrieve other texts that are also help the RAG pipeline generate good answers. We therefore also use an LLM to evaluate answer quality more broadly.

In [10]:
# create an huggingface inference endpoint to run any LLM
# intro: https://www.philschmid.de/inference-endpoints-iac
# docs: https://huggingface.co/docs/huggingface_hub/v0.20.1/en/package_reference/hf_api#huggingface_hub.HfApi.create_inference_endpoint
from huggingface_hub import create_inference_endpoint
from huggingface_hub import HfApi
api = HfApi()


# define TGI as custom image
custom_image = {
    "health_route": "/health",  # Health route for TGI
    "env": {
        "MAX_BATCH_PREFILL_TOKENS": "2048", # can be adjusted to your needs
        "MAX_INPUT_LENGTH": "1024", # can be adjusted to your needs
        "MAX_TOTAL_TOKENS": "1512", # can be adjusted to your needs
        "MODEL_ID": "/repository",  # IE will save the model in /repository
    },
    "url": "ghcr.io/huggingface/text-generation-inference:1.3.3",
}

# Create Inference Endpoint to run Zephyr 7B
print("Creating Inference Endpoint for Zephyr 7B")
hf_endpoint = create_inference_endpoint(
    name="zehpyr-ie-test",
    repository="HuggingFaceH4/zephyr-7b-beta",
    framework="pytorch",
    task="text-generation",
    vendor="aws",
    region="us-east-1",
    type="protected",
    instance_size="medium",
    instance_type="g5.2xlarge",  # A10G GPU. Pricing: https://aws.amazon.com/sagemaker/pricing/?p=pm&c=sm&z=4
    accelerator="gpu",
    namespace="MoritzLaurer",  # your user or organisation name on the HF hub
    custom_image=custom_image,
)

print("Waiting for endpoint to be deployed")
hf_endpoint.wait()

print("Endpoint ready")

# to manage an existing endpoint, use:
#hf_endpoint = api.get_inference_endpoint("zehpyr-ie-test")
#hf_endpoint.resume()
#hf_endpoint.pause()
#hf_endpoint.delete()

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import HuggingFaceEndpoint
from langchain.prompts import ChatPromptTemplate, PromptTemplate

provider_for_question_generation = "HF"


if provider_for_question_generation == "HF":
  chat_model = HuggingFaceEndpoint(
    endpoint_url=hf_endpoint.url,  #"https://ytjpei7t003tedav.us-east-1.aws.endpoints.huggingface.cloud",
    task="text-generation",
    huggingfacehub_api_token=userdata.get('HF_TOKEN'),
    model_kwargs={}
  )

elif provider_for_question_generation == "OAI":
  # https://platform.openai.com/docs/api-reference/chat
  chat_model = ChatOpenAI(
      model="gpt-3.5-turbo-1106",  #"gpt-3.5-turbo-1106",  # "gpt-4-1106-preview"
      temperature=0.2, max_tokens=1024,
      n=1, top_p=0.95,
      frequency_penalty=0.0,  # Number between -2.0 and 2.0. Positive values penalize new tokens based on their existing frequency in the text so far, decreasing the model's likelihood to repeat the same line verbatim.
      presence_penalty=0.0,  # Number between -2.0 and 2.0. Positive values penalize new tokens based on whether they appear in the text so far, increasing the model's likelihood to talk about new topics.
      #response_format={ "type": "json_object" },
      seed=42,
  )



In [12]:

instruction_question_gen = """\
Your task is to write a factoid question given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine. \
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

context: {context}\n
factoid question: """


prompt_question_gen = ChatPromptTemplate.from_template(instruction_question_gen)

chain = prompt_question_gen | chat_model

questions_lst = []
for context in docs_processed_for_q_generation:
  print("Context:\n", context.page_content)
  output_question = chain.invoke({"context": context.page_content})
  if provider_for_question_generation == "OAI":
    output_question = output_question.content
  print("\nGenerated question:\n", output_question, "\n")
  questions_lst.append(output_question)


Context:
 access to school-made teaching material covering the achieved or 
partially achieved curriculum. Therefore teachers need telecom 
support and home-working status as well as authorship 
recognition. 
"A School", as we put it, "is a seamless process for transferring 
knowledge and experimenting newly acquired knowledge." 
From the user point of view, this is a crucial issue. For Education 
Bodies, it is a critical mission. For multimedia technologists, it is a 
real challenge. 
As Professor Ruberti used to say: "Information Technology and 
Telecommunications are the only ways today to help teachers 
upgrading the Education System's performances." 
Are all the players in this area convinced that they have to 
proceed this way ? 
What do think of it our newest regulatory bodies? 
Are they ready to enforce the introduction of Isdn as a bottom 
line for the Universal Service definition ? 
And what about our governments ? 
Would it be possible to ask them yearly accounts of their

G

## RAG pipeline

### Retrival

Optimization potential: different retrievers, different rerankers, multi-retrievers

In [13]:
# detailed RAG docs: https://python.langchain.com/docs/use_cases/question_answering/
# FAISS cookbook: https://python.langchain.com/docs/expression_language/cookbook/retrieval
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings, HuggingFaceInferenceAPIEmbeddings
from langchain.schema import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma, Qdrant
from langchain_core.runnables import RunnablePassthrough
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams


In [14]:
# ! issue: langchain vector store wrappers don't seem to allow adjustment to dimensions, only accept OAI default 1.5k
# using qdrant directly instead of langchain wrapper

provider_retrieval_model = "HF"

client_path = f"./vectorstore"
collection_name = f"collection"

if provider_retrieval_model == "HF":
  qdrantClient = QdrantClient(path=client_path, prefer_grpc=True)

  embeddings = HuggingFaceInferenceAPIEmbeddings(
      api_key=userdata.get('HF_TOKEN'), model_name="sentence-transformers/all-MiniLM-l6-v2"
  )

  dim = 384

elif provider_retrieval_model == "OAI":

  qdrantClient = QdrantClient(path=client_path, prefer_grpc=True)

  embeddings = OpenAIEmbeddings(
          model="text-embedding-ada-002",
          openai_api_key=os.getenv("OPENAI_API_KEY"),
  )

  dim = 1536


qdrantClient.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=dim, distance=Distance.COSINE),
)

vectorstore = Qdrant(
    client=qdrantClient,
    collection_name=collection_name,
    embeddings=embeddings,
)

vectorstore.add_documents(docs_processed_samp)

['28c3c99dfb1949a4a86d9833ce55fc5e',
 '69ec7cd463b947c7970f27d071b70bdc',
 '3d4653e0b63c4a498b4a54fa43a3d6c2',
 '2b31dd2ee13b48a1aac6518dadd67314',
 '4d2cbb5f53194e729e8b9497ca9dc3f6',
 '99ae290808e44bb09bd008c2e649898c',
 '7629e15f8d8745a6ab89349386457186',
 '62574f2f9e4449179b2715c69510e2cd',
 '99d38b296a0541d0acb6e330fcd58e48',
 '181e9dd9378848528c66a162138ad7df',
 '34a3e74b37c7448d8cf00fcab3fa6ba9',
 '68f8cb9563f842acb69549ef148e607f',
 '5ce8ac570ee546e6ae5b279fbb7c31dc',
 '08d75ae58032475789f67a8cee497340',
 '046cda3cd10e4f3fa6c2c72107ae595e',
 '4e9e717e653d4530b62755e8097c4df6',
 '617806d2d540448983428a329dade2d5',
 '757fc890e6c54e00833be22a9c2b2453',
 '7af6c7d36fc246bfa221c3f8be9088bf',
 '27d5a0c3879844689da7d0fcd1744ee7',
 '80d26b5ac016439c8c9ca52271d8436e',
 '42a1683ebc114b3dbb3bb2528038fa58',
 '56970c1f3c824acba4d60a118416c580',
 '6904e3761d8145b7ad53bc667145552d',
 '8ab4c34263454c0bb63107067cabd45d',
 'bf5a13231d3640a28339da855901aa56',
 '26ba9b4b01324365afa9e7048766ac16',
 

In [15]:

context_retrieved_lst = []
for question in questions_lst:
  retriever = vectorstore.as_retriever(
      search_type="similarity",
      search_kwargs={"k": 1}
  )

  context_retrieved = retriever.get_relevant_documents(
      question
  )

  def format_docs(docs):
      return "\n\n".join(doc.page_content for doc in docs)

  context_retrieved = format_docs(context_retrieved)

  context_retrieved_lst.append(context_retrieved)
  #print(context_retrieved)


In [16]:
# check if retrieved context for question is same as context used for generating the question
# note that this is an imperfect measure, because the retriever might
# retrieve other texts that are equally relevant as the text used for generating the question
context_for_q_generation = [doc.page_content for doc in docs_processed_for_q_generation]
correct_context_retrieved = [a == b for a, b in zip(context_for_q_generation, context_retrieved_lst)]

retrieval_accuracy = sum(correct_context_retrieved) / len(correct_context_retrieved)
print(retrieval_accuracy)


0.8


In [17]:
# add reranking step
# challenge: reranking with HF models not implemented in langchain
# only cohere reranker seems implemented: https://python.langchain.com/docs/integrations/retrievers/cohere-reranker

"""import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-base')
model = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-base')
model.eval()

context_question_pairs_lst = []
for question in questions_lst:
  context_question_pairs_lst.append([[question, context] for context in context_retrieved_lst])

#pairs = [['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']]

for context_question_pair in context_question_pairs_lst:
  with torch.no_grad():
      inputs = tokenizer(context_question_pair, padding=True, truncation=True, return_tensors='pt', max_length=512)
      scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
      print(scores)
"""

"import torch\nfrom transformers import AutoModelForSequenceClassification, AutoTokenizer\n\ntokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-base')\nmodel = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-base')\nmodel.eval()\n\ncontext_question_pairs_lst = []\nfor question in questions_lst:\n  context_question_pairs_lst.append([[question, context] for context in context_retrieved_lst])\n\n#pairs = [['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']]\n\nfor context_question_pair in context_question_pairs_lst:\n  with torch.no_grad():\n      inputs = tokenizer(context_question_pair, padding=True, truncation=True, return_tensors='pt', max_length=512)\n      scores = model(**inputs, return_dict=True).logits.view(-1, ).float()\n      print(scores)\n"

### Answer generation

Optimization potential: different LLMs, different prompt templates

In [18]:

prompt_qa_template = """\
Your task is to answer a question based on a context.
Your answer should be concise and you should only return your answer.

context: {context}
question: {question}
answer: """

prompt_qa_template = PromptTemplate.from_template(prompt_qa_template)


In [19]:
from langchain.llms import HuggingFaceEndpoint

provider_answer_model = "HF"

if provider_answer_model == "HF":
  llm_qa = HuggingFaceEndpoint(
    endpoint_url=hf_endpoint.url,  #"https://nqoa2is3qe7y82ww.us-east-1.aws.endpoints.huggingface.cloud",
    task="text-generation",
    huggingfacehub_api_token=userdata.get('HF_TOKEN'),
    model_kwargs={}
  )
elif provider_answer_model == "OAI":
  llm_qa = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

chain = prompt_qa_template | llm_qa | StrOutputParser()

answer_lst = []
for question, context in zip(questions_lst , context_retrieved_lst):
  answer = chain.invoke({"context": context, "question": question})
  answer_lst.append(answer)


### Automatic LLM evaluation of generated answer

In [20]:
# this scoring prompt can be freely adapted to evaluation criteria
# of different use-cases

instruction_judge_answer = """\
Your task is to score the quality of an answer to a question in a given context.

Your scoring criteria for assessing the answer are:
- pertinence: Does the answer directly answer the question?
- context grounding: Is the answer clearly grounded in the context? To be well grounded, the answer does not need to explicitly reference the context.
- conciseness: Is the answer concise without unnecessary verbosity?

Your quality score should be in the range of 0 to 100.\
100 means a very good answer, 0 means a very bad answer, 50 means a mediocre answer.

First briefly reason step-by-step to assess the extent to which the answer fulfills these criteria. Your reasoning should be short.
Then return the quality score.

Always answer in this JSON evaluation format: {{"reason": "...", "score": "..."}}

context: {context}\n
question: {question}\n
answer: {answer}\n
JSON evaluation: """

instruction_judge_answer = ChatPromptTemplate.from_template(instruction_judge_answer)

# currently need to use OAI here, because it enforces JSON very well
llm_evaluation = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

chain = instruction_judge_answer | llm_evaluation


output_quality_lst = []
for answer, question, context_retrieved in zip(answer_lst, questions_lst, context_retrieved_lst):

  output_quality = chain.invoke({
      "context": context_retrieved,
      "question": question,
      "answer": answer
  })

  output_quality_lst.append(output_quality.content)



In [21]:
# parsing the JSON output can lead to errors
# with open-source models, which don't enforce JSON as well as OAI
import ast

output_quality_dic = [ast.literal_eval(output) for output in output_quality_lst]
output_quality_score = [int(dic["score"]) for dic in output_quality_dic]
output_quality_reason = [dic["reason"] for dic in output_quality_dic]


## Results

In [22]:
import pandas as pd

df_results = pd.DataFrame({
    "question": questions_lst,
    "answer": answer_lst,
    "answer_quality_score": output_quality_score,
    "answer_quality_reason": output_quality_reason,
    "correct_context": [a == b for a, b in zip(context_for_q_generation, context_retrieved_lst)],
    "context_retrieved": context_retrieved_lst,
    "context_for_q_generation": context_for_q_generation
})

mean_answer_score = df_results["answer_quality_score"].mean()
retrieval_accuracy = sum(df_results["correct_context"]) / len(df_results["correct_context"])

print(f"Retrieval accuracy: {retrieval_accuracy}")
print(f"Mean answer socre: {mean_answer_score}")
print("\n")

df_results


Retrieval accuracy: 0.8
Mean answer socre: 90.0




,question,answer,answer_quality_score,answer_quality_reason,correct_context,context_retrieved,context_for_q_generation
0,\nWhat is the role of Information Technology a...,"\nAccording to Professor Ruberti, the roles of...",70,The answer directly answers the question by st...,True,access to school-made teaching material coveri...,access to school-made teaching material coveri...
1,"\n""What percentage of engineers say they would...",48% of engineers say they would benefit from t...,80,The answer directly answers the question and i...,True,got materials engineering that’s coming in. An...,got materials engineering that’s coming in. An...
2,\nWhat is the recommended approach for ensurin...,\nThe recommended approach for ensuring the pr...,100,The answer directly answers the question by st...,False,"including rules on the burden of proof, should...",wymogów należy ograniczyć do zastosowań SI wys...
3,\n\nwhat is the legal stance of deployers rega...,\n\nDeployers in the given context can often a...,100,The answer directly answers the question by st...,True,deployer can often absolve themselves for any ...,deployer can often absolve themselves for any ...
4,"\n\nWhat is the EU's stance on regulating AI, ...",\n\nThe EU's stance on regulating AI is to all...,100,The answer directly answers the question by ex...,True,should allow the benefits of AI to be realized...,should allow the benefits of AI to be realized...
